# 04 - RAG QA

学习检索增强生成（RAG）技术，通过文档检索来提高问答的准确性。

## 学习目标
- 理解 RAG 的基本原理和优势
- 掌握文档加载和预处理
- 学习向量化存储和检索
- 实现完整的 RAG 问答系统

In [ ]:
# 导入必要的库
import os
from dotenv import load_dotenv
from pydantic import SecretStr

# 加载环境变量
load_dotenv(override=True)

# 检查 API 密钥
if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("请设置 OPENAI_API_KEY 环境变量")

# 导入 LangChain 组件
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import TextLoader
from langchain_core.prompts import ChatPromptTemplate

print("✓ 环境和组件导入完成")

## 1. 准备文档数据

In [ ]:
# 创建示例文档（如果没有网络，使用本地文本）
sample_docs = [
    """
Python 是一种高级编程语言，由 Guido van Rossum 于 1991 年首次发布。
Python 具有简洁明了的语法，易于学习和使用，被广泛应用于 Web 开发、
数据科学、人工智能、自动化脚本等领域。

Python 的主要特点包括：
- 语法简洁，可读性强
- 支持多种编程范式（面向对象、函数式、过程式）
- 丰富的标准库和第三方库
- 跨平台，可在多种操作系统上运行
- 活跃的社区支持
""",
    """
机器学习是人工智能的一个重要分支，它使计算机能够在没有明确编程的情况下
学习和改进。机器学习算法通过分析数据来识别模式，并基于这些模式做出预测或决策。

机器学习的主要类型包括：
1. 监督学习：使用标记的训练数据学习输入和输出之间的映射关系
2. 无监督学习：从无标记的数据中发现隐藏的模式和结构
3. 强化学习：通过与环境交互学习最优策略

常见的机器学习算法有线性回归、决策树、神经网络、支持向量机等。
""",
    """
LangChain 是一个用于构建基于大语言模型应用程序的框架。它提供了一套工具和组件，
帮助开发者更容易地创建复杂的 AI 应用。

LangChain 的核心功能包括：
- 模型抽象：统一不同 LLM 提供商的接口
- 提示词管理：创建和管理复杂的提示词模板
- 链式调用：将多个组件串联成工作流
- 记忆管理：为对话系统添加记忆功能
- 智能体：创建能够使用工具的自主智能体
- 索引和检索：构建 RAG 系统

LangChain 支持 Python 和 JavaScript/TypeScript，是目前最受欢迎的 LLM 应用开发框架之一。
"""
]

print("✓ 示例文档准备完成")

In [ ]:
# 保存文档到临时文件（用于演示）
os.makedirs("temp_docs", exist_ok=True)

doc_files = [
    ("temp_docs/python_intro.txt", sample_docs[0]),
    ("temp_docs/ml_intro.txt", sample_docs[1]),
    ("temp_docs/langchain_intro.txt", sample_docs[2])
]

for file_path, content in doc_files:
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(content.strip())

print("✓ 文档文件创建完成")

## 2. 文档加载和预处理

In [ ]:
# 加载文档
loader = TextLoader("temp_docs/python_intro.txt", encoding='utf-8')
documents = loader.load()

# 加载所有文档
all_documents = []
for file_path, _ in doc_files:
    loader = TextLoader(file_path, encoding='utf-8')
    docs = loader.load()
    all_documents.extend(docs)

print(f"✓ 加载了 {len(all_documents)} 个文档")
for i, doc in enumerate(all_documents):
    print(f"  文档 {i+1}: {doc.metadata.get('source', 'unknown')} ({len(doc.page_content)} 字符)")

In [ ]:
# 文档分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 每个分块的最大字符数
    chunk_overlap=50,  # 分块之间的重叠字符数
    separators=["\n\n", "\n", " ", ""]  # 分割优先级
)

splits = text_splitter.split_documents(all_documents)

print(f"✓ 文档分割完成，共 {len(splits)} 个分块")
for i, split in enumerate(splits[:3]):  # 显示前 3 个分块
    print(f"\n--- 分块 {i+1} ---")
    print(f"来源：{split.metadata.get('source', 'unknown')}")
    print(f"内容预览：{split.page_content[:100]}...")

## 3. 向量化存储

In [ ]:
# 从环境变量读取配置
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_BASE_URL")

# 初始化 OpenAI 嵌入模型
embeddings = OpenAIEmbeddings(api_key=SecretStr(api_key), base_url=base_url)

print("✓ OpenAI 嵌入模型初始化完成")

In [ ]:
# 创建向量数据库
# 使用 Chroma 作为向量存储
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="./chroma_db"  # 持久化存储目录
)

print("✓ 向量数据库创建完成")
print(f"存储了 {len(splits)} 个文档分块")

## 4. 测试检索功能

In [ ]:
# 测试相似性搜索
query = "Python 的特点是什么？"

# 检索相关文档
relevant_docs = vectorstore.similarity_search(query, k=3)  # 返回最相关的 3 个文档

print(f"查询：{query}")
print(f"\n找到 {len(relevant_docs)} 个相关文档：")

for i, doc in enumerate(relevant_docs):
    print(f"\n--- 文档 {i+1} ---")
    print(f"来源：{doc.metadata.get('source', 'unknown')}")
    print(f"内容：{doc.page_content}")

In [ ]:
# 测试更多查询
test_queries = [
    "机器学习有哪些类型？",
    "LangChain 的核心功能是什么？",
    "如何学习编程？"
]

for query in test_queries:
    print(f"\n=== 查询：{query} ===")
    docs = vectorstore.similarity_search(query, k=2)
    
    for i, doc in enumerate(docs):
        print(f"\n文档 {i+1}：")
        print(f"{doc.page_content[:150]}...")

## 5. 创建 RAG 问答系统

In [ ]:
# 从环境变量读取配置
model_name = os.getenv("MODEL_NAME", "gpt-3.5-turbo")

# 初始化 LLM
llm = ChatOpenAI(
    model=model_name,
    temperature=0,
    api_key=SecretStr(api_key),
    base_url=base_url,
)

print("✓ LLM 初始化完成")

In [ ]:
# 创建自定义提示词模板
prompt_template = ChatPromptTemplate.from_template(
    """基于以下上下文信息回答问题。如果上下文中没有相关信息，请说"根据提供的文档，我无法回答这个问题"。

上下文：
{context}

问题：{input}

请提供准确、详细的回答："""
)

print("✓ RAG 提示词模板创建完成")

In [ ]:
# 创建 RAG 链
# 创建文档链
document_chain = create_stuff_documents_chain(llm, prompt_template)

# 创建检索链
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}  # 检索前 3 个最相关的文档
)
qa_chain = create_retrieval_chain(retriever, document_chain)

print("✓ RAG 问答链创建完成")

## 6. 测试 RAG 问答

In [ ]:
# 测试 RAG 问答
questions = [
    "Python 有哪些主要特点？",
    "机器学习的主要类型有哪些？",
    "LangChain 提供哪些核心功能？",
    "如何学习人工智能？",  # 这个问题可能无法直接回答
    "Python 是谁创建的？"
]

for question in questions:
    print(f"\n=== 问题：{question} ===")
    
    # 获取回答
    result = qa_chain.invoke({"input": question})
    
    print(f"回答：{result['answer']}")
    
    # 显示源文档
    if 'context' in result:
        print("\n参考来源：")
        for i, doc in enumerate(result['context']):
            source = doc.metadata.get('source', 'unknown')
            print(f"  {i+1}. {source}")

## 7. 进阶功能

In [ ]:
# 演示如何添加新文档
new_doc_content = """
深度学习是机器学习的一个子领域，它使用多层神经网络来学习数据的复杂模式。
深度学习在图像识别、自然语言处理、语音识别等领域取得了突破性进展。

常见的深度学习框架包括 TensorFlow、PyTorch、Keras 等。
这些框架提供了构建和训练神经网络的工具和库。
"""

# 保存新文档
with open("temp_docs/deep_learning.txt", 'w', encoding='utf-8') as f:
    f.write(new_doc_content.strip())

# 加载并分割新文档
new_loader = TextLoader("temp_docs/deep_learning.txt", encoding='utf-8')
new_docs = new_loader.load()
new_splits = text_splitter.split_documents(new_docs)

# 添加到向量数据库
vectorstore.add_documents(new_splits)

print(f"✓ 添加了 {len(new_splits)} 个新的文档分块")

In [ ]:
# 测试新添加的内容
new_questions = [
    "什么是深度学习？",
    "有哪些深度学习框架？",
    "深度学习和机器学习的关系是什么？"
]

for question in new_questions:
    print(f"\n=== 问题：{question} ===")
    result = qa_chain.invoke({"input": question})
    print(f"回答：{result['answer']}")

## 8. 性能分析

In [ ]:
# 分析检索质量
def analyze_retrieval(query, k=3):
    """分析检索结果的质量"""
    docs = vectorstore.similarity_search_with_score(query, k=k)
    
    print(f"查询：{query}")
    print(f"\n检索结果（按相似度排序）：")
    
    for i, (doc, score) in enumerate(docs):
        print(f"\n{i+1}. 相似度分数：{score:.4f}")
        print(f"   来源：{doc.metadata.get('source', 'unknown')}")
        print(f"   内容：{doc.page_content[:100]}...")

# 测试不同查询的检索效果
test_queries_for_analysis = [
    "Python 编程语言",
    "神经网络",
    "AI 应用开发"
]

for query in test_queries_for_analysis:
    print("\n" + "="*50)
    analyze_retrieval(query)

## 9. 清理资源

In [ ]:
# 清理临时文件（可选）
import shutil

def cleanup():
    """清理临时文件和数据库"""
    try:
        # 删除临时文档
        if os.path.exists("temp_docs"):
            shutil.rmtree("temp_docs")
            print("✓ 临时文档已清理")
        
        # 删除向量数据库（可选）
        # if os.path.exists("chroma_db"):
        #     shutil.rmtree("chroma_db")
        #     print("✓ 向量数据库已清理")
        
    except Exception as e:
        print(f"清理过程中出现错误：{e}")

# 取消注释以运行清理
# cleanup()

print("\n🎉 RAG QA 示例运行完成！")
print("提示：可以手动运行 cleanup() 函数清理临时文件")

## 总结

在这个示例中，我们学习了：

1. **文档加载**: 使用 TextLoader 加载本地文档
2. **文档分割**: RecursiveCharacterTextSplitter 将长文档分割成适当大小的块
3. **向量化**: OpenAIEmbeddings 将文本转换为向量表示
4. **向量存储**: Chroma 向量数据库存储和检索文档
5. **RAG 链**: 使用 create_retrieval_chain 组合检索和生成功能
6. **自定义提示**: 设计适合 RAG 的提示词模板
7. **动态添加**: 支持运行时添加新文档
8. **性能分析**: 检查检索质量和相似度分数

### 关键概念
- **Embeddings**: 文本的数值表示，用于计算相似度
- **Vector Store**: 存储和检索向量的数据库
- **Retriever**: 根据查询检索相关文档的组件
- **Chunk Size**: 文档分块的大小，影响检索精度和性能
- **Overlap**: 分块间的重叠，确保上下文连续性

### 最佳实践
1. **分块大小**: 500-1000 字符通常效果较好
2. **重叠设置**: 10-20% 的分块大小作为重叠
3. **检索数量**: k=3-5 通常足够
4. **提示词设计**: 强调基于上下文回答
5. **质量检查**: 定期检查检索结果的相关性

### 应用场景
- 知识库问答系统
- 文档检索和总结
- 客服支持系统
- 学术研究辅助
- 个人知识管理

### 下一步
尝试将 RAG 与记忆、智能体等功能结合，构建更复杂的 AI 应用！